<a href="https://colab.research.google.com/github/GuidoAH/Optimization-Algorithms/blob/main/Algoritmos_Guido_Alexander_Heienbrok_AG3_V_Extendida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3
# Guido Alexander Heienbrok
# Link GitHub: https://github.com/GuidoAH/Optimization-Algorithms

#Indice

1. Cargar datos del problema
2. Funciones básicas
3. Búsqueda aleatoria
4. Búsqueda local<br>
  4.1.1 Búsqueda Local Extendida

In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


## 1. Cargar los datos del problema

In [2]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

In [3]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [4]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

# 2. Funciones básicas

In [5]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

# 3. Búsqueda aleatoria

In [6]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 14, 37, 16, 6, 5, 1, 18, 19, 15, 13, 26, 39, 23, 34, 25, 7, 32, 33, 22, 17, 36, 35, 28, 8, 41, 9, 30, 10, 24, 38, 27, 12, 21, 29, 2, 31, 4, 3, 20, 40, 11]
Distancia     : 3737


# 4. Búsqueda local

In [7]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3737
Distancia Mejor Solucion Local: 3419


In [8]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  print(distancia_total(solucion_referencia,problem))

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem)

5093
En la iteracion  26 , la mejor solución encontrada es: [0, 37, 15, 16, 14, 19, 13, 28, 29, 30, 22, 9, 8, 26, 5, 18, 12, 11, 2, 27, 3, 4, 10, 23, 41, 25, 6, 7, 17, 31, 20, 38, 24, 40, 21, 39, 36, 35, 33, 34, 32, 1]
Distancia     : 2061


## 4.1.1 Búsqueda Local Extendida

A continuación se profundiza la búsqueda local, anadiendo un multiarranque a la búsqueda local. Es decir, el proceso sería el siguiente:
- Se genera una solución aleatoria
- Se busca la mejor solución vecina y se guarda esa mejor solución como "ultra_mejor_solucion".
- A continuación se vuelve a crear una solución aleatoria, se vuelve a buscar la mejor solución vecina. Sí la nueva solución vecina es mejor que "ultra_mejor_solucion", entonces la nueva mejor solución vecina pasa a ser "ultra_mejor_solucion"

En definita, el algoritmo de búsqueda local anterior se ha anidado dentro de otro bucle que dependerá de un número de iteraciones.


In [10]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  iterator1 = 0
  while iterator1 < 20:
    #Generar una solucion inicial de referencia(aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)
    if iterator1 == 0:
      ultra_mejor_distancia = mejor_distancia
      ultra_mejor_solucion = solucion_referencia
#    print(distancia_total(solucion_referencia,problem))

    iterator2=0             #Un contador para saber las iteraciones que hacemos
    while(1):
      iterator2 +=1         #Incrementamos el contador
      #print('#',iteracion)

      #Obtenemos la mejor vecina ...
      vecina = genera_vecina(solucion_referencia)

      #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
      distancia_vecina = distancia_total(vecina, problem)

      #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina                   #Guarda la mejor solución encontrada
        mejor_distancia = distancia_vecina

      else:
#        print(f"Mejor solución local {mejor_solucion}")
        break

      solucion_referencia = vecina

    if mejor_distancia < ultra_mejor_distancia:
      ultra_mejor_distancia = mejor_distancia
      ultra_mejor_solucion = mejor_solucion

    iterator1 += 1
    print("Distancia:" , ultra_mejor_distancia)


sol = busqueda_local(problem)

Distancia: 1781
Distancia: 1781
Distancia: 1781
Distancia: 1709
Distancia: 1709
Distancia: 1709
Distancia: 1709
Distancia: 1709
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1647
Distancia: 1588
